In [ ]:
import load_data as ld
import re

In [186]:
c.load_data("twitter-datasets/train_neg_full.txt", "twitter-datasets/train_pos_full.txt", "twitter-datasets/test_data.txt")

AttributeError: module 'code' has no attribute 'load_data'

In [159]:
train_path_neg = "twitter-datasets/train_neg_full.txt"
train_path_pos = "twitter-datasets/train_pos_full.txt"
test_path = "twitter-datasets/test_data.txt"

In [161]:
X_train, y_train, X_test = c.load_data(train_path_neg, train_path_pos, test_path)

In [162]:
X_train.head()

0                          stronger without youu <url>
1                                 <user> thanks hun xx
2    <user> ohhh gurlll ! woop ! and i know , it's ...
3    <user> you should see and hear me right now it...
4    <user> i might get some made , limited run usi...
Name: tweet, dtype: object

In [ ]:
def preprocess_tweet(tweet):
    return tweet

In [ ]:
def preprocess_word(word):
    word 
    return word

In [169]:
def preprocess_data(X_train, X_test):
    
    X_train = X_train.apply(str).apply(str.lower)
    X_test = X_test.apply(str).apply(str.lower)
    
    patterns = [
        (re.compile(r'<[^>]*>'), ' '), # HTML tags
        (re.compile(r'([a-zA-Z])\1{2,}'), '') # Letters more than three times in a row
    ]
    
    for pattern, replacement in patterns:
        X_train = X_train.apply(lambda x: pattern.sub(replacement, x))
        X_test = X_test.apply(lambda x: pattern.sub(replacement, x))
        
    return X_train, X_test

In [170]:
X_train, X_test = preprocess_data(X_train, X_test)

In [171]:
X_train.head()

0                              stronger without youu  
1                                        thanks hun xx
2      o gur ! woop ! and i know , it's either chil...
3      you should see and hear me right now it's ho...
4      i might get some made , limited run using   ...
Name: tweet, dtype: object